- https://medium.com/@hammadhaideerr/how-to-quantize-a-language-model-to-int8-using-optimum-2fa306767338

In [1]:
from transformers import DistilBertModel, DistilBertTokenizer

# Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
model = DistilBertModel.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

/Users/owner/My Drive/Hassan USB Data/Bytewise NLP/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/owner/My Drive/Hassan USB Data/Bytewise NLP/env/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/owner/My Drive/Hassan USB Data/Bytewise NLP/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import torch

# Export the model to ONNX with opset 14
onnx_model_path = "Dynamic Quantization/distilbert_base_uncased.onnx"
dummy_input = torch.randint(0, 100, (1, 10))  # Dummy input for ONNX export
torch.onnx.export(model, dummy_input, onnx_model_path, opset_version=14)

In [4]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Perform dynamic quantization on the exported ONNX model
quantized_model_path = "Dynamic Quantization/quantized_distilbert_base_uncased.onnx"
quantize_dynamic(onnx_model_path, quantized_model_path, weight_type=QuantType.QInt8)

Ignore MatMul due to non constant B: /[/transformer/layer.0/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.0/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.1/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.1/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.2/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.2/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.3/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.3/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.4/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.4/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.5/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.5/attention/MatMul_1]


In [5]:
import onnx

# Load the quantized model to verify it was saved correctly
quantized_model = onnx.load(quantized_model_path)
onnx.checker.check_model(quantized_model)

print(f"Quantized DistilBERT model saved to {quantized_model_path}")

Quantized DistilBERT model saved to Dynamic Quantization/quantized_distilbert_base_uncased.onnx


- https://medium.com/@shehrozaslam1317/converting-a-language-model-to-int8-using-optimum-56e318535424

In [6]:
from transformers import DistilBertModel, DistilBertTokenizer

# Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
model = DistilBertModel.from_pretrained(model_name)

/Users/owner/My Drive/Hassan USB Data/Bytewise NLP/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
model.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.33.3",
  "vocab_size": 30522
}

In [17]:
import torch

onnx_model_path = "Dynamic Quantization 2/model.onnx"
dummy_input = (
    torch.randint(0, 100, (1, 10)),  # input_ids
    torch.randint(0, 2, (1, 10))  # attention_mask
)

# Export the model to ONNX
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=14
)

/Users/owner/My Drive/Hassan USB Data/Bytewise NLP/env/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [18]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantized_model_path = "Dynamic Quantization 2/Quantized_model.onnx"
# Perform dynamic quantization
quantize_dynamic(
    onnx_model_path,
    quantized_model_path,
    weight_type=QuantType.QInt8
)

Ignore MatMul due to non constant B: /[/transformer/layer.0/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.0/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.1/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.1/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.2/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.2/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.3/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.3/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.4/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.4/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layer.5/attention/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layer.5/attention/MatMul_1]


In [23]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer

# Load the quantized ONNX model
session = InferenceSession(quantized_model_path, providers=['AzureExecutionProvider', 'CPUExecutionProvider'])

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the input data
inputs = tokenizer("This is a test sentence", return_tensors="pt", max_length=10, padding="max_length", truncation=True)
input_ids_np = inputs["input_ids"].numpy()
attention_mask_np = inputs["attention_mask"].numpy()

# Run Inference
outputs = session.run(None, {"input_ids": input_ids_np, "attention_mask": attention_mask_np})

print(outputs)

[array([[[-0.2679795 , -0.11143361, -0.01467639, ..., -0.14686449,
          0.15257041,  0.4456752 ],
        [-0.61722964, -0.4335706 , -0.03758816, ..., -0.23236962,
          0.46683633,  0.3649221 ],
        [-0.41510642, -0.29438594,  0.10258104, ..., -0.05495847,
          0.07371355,  0.6923371 ],
        ...,
        [-0.22827029, -0.12904058,  0.08350199, ...,  0.07397674,
          0.06351829,  0.31308138],
        [-0.14275408, -0.13677111,  0.22930504, ...,  0.02308208,
          0.11602733,  0.22510473],
        [-0.17076543, -0.17607294,  0.10926739, ...,  0.15501772,
         -0.01684695,  0.4110286 ]]], dtype=float32)]
